# 1. Define imports

In [160]:
import librosa
import librosa.display
import IPython.display as ipd
import numpy as np
import csv
import pandas as pd
from tensorflow.keras.models import load_model
import joblib

# 2. Load Model 

In [161]:
# Deepfake Detection
df_model = load_model('models/DeepFake_model_ver4_full.keras')

In [162]:
print(df_model)

<Sequential name=sequential_1, built=True>


In [189]:
# Speaker Identification
si_model = load_model('models/speaker_identification_model_ver4_segment.keras')

In [174]:
print(si_model)

<Sequential name=sequential_2, built=True>


# 3. Evaluation

In [190]:
def reshape_segment(segment_features):
    # Load the scaler
    df_scaler = joblib.load('scalers/df_scaler.pkl') # Deepfake Detection
    si_scaler = joblib.load('scalers/vr_scaler.pkl') # Speaker Identification
    
    # Scale the features using the pre-fitted scaler
    # reshaped = df_scaler.transform([segment_features])  # Deepfake Detection
    reshaped = si_scaler.transform([segment_features])  # Speaker Identification
    
    # Reshape to match the model's input shape (num_features, 1, 1)
    reshaped = reshaped.reshape(1, reshaped.shape[1], 1, 1)  # Shape: (1, num_features, 1, 1)
    print(reshaped.shape)
    return reshaped

In [191]:
def evaluate_segments(segment_data):
    confidence_scores = []  # Store the scores for each segment
    # iterate each row
    for i in range(0, segment_data.shape[0]):
        # Reshape the segment features to match the model's input
        segment_row = segment_data[i]
        
        reshaped_segment = reshape_segment(segment_row)
        
        # Predict the confidence score using the model
        # predictions = df_model.predict(reshaped_segment) # Deepfake Detection
        predictions = si_model.predict(reshaped_segment) # Speaker Identification
    
        confidence = predictions[0][0]
        
        # Store the confidence score
        confidence_scores.append(confidence)

    return confidence_scores


# 4. Load single data

In [201]:
# Deepfake Detection test
# segment_data = pd.read_csv('result\deepfake_detection\\tests\human_t1.csv') # human
# segment_data = pd.read_csv('result\deepfake_detection\\tests\human_t2.csv') # human
# segment_data = pd.read_csv('result\deepfake_detection\\tests\synthetic_t1.csv') # synthetic
# segment_data = pd.read_csv('result\deepfake_detection\\tests\synthetic_t2.csv') # synthetic

# Speaker Identification test
# segment_data = pd.read_csv('result\deepfake_detection\\tests\human_t2.csv') # same
segment_data = pd.read_csv('result\\voice_recognition\\test_full\\vr_full_segment_same.csv') # same

"""

Output: 
Deepfake Detection = 1 Human : 0 AI
Speaker Identification = 1 Speaker : 0 Not Speaker

"""

print(segment_data)
segment_data = segment_data.drop(columns=['file_name']).values
confidence_scores = evaluate_segments(segment_data)

                  file_name  mfcc_feature1  mfcc_feature2  mfcc_feature3  \
0  Testspeaker_notonDS3.mp3      -466.5779       97.13960     -26.258076   
1  Testspeaker_notonDS3.mp3      -417.8303      132.05717     -34.885048   
2  Testspeaker_notonDS3.mp3      -367.9756      157.65352     -55.006313   

   mfcc_feature4  mfcc_feature5  mfcc_feature6  mfcc_feature7  mfcc_feature8  \
0      27.655281       6.587523       3.792437      -4.188441      -5.875909   
1      61.646320       1.144908      13.338906      -0.824676      -3.358345   
2      59.517086      -2.079819       5.414917      -7.052460       9.160208   

   mfcc_feature9  ...  SpectralBandwidth78  SpectralBandwidth79  \
0      -2.479454  ...          1167.923996          1100.285335   
1      -9.944326  ...          2281.718069          2043.592242   
2      -8.061013  ...          1377.506934          1340.238107   

   SpectralBandwidth80  SpectralBandwidth81  SpectralBandwidth82  \
0           873.838629           975.

In [202]:
print(confidence_scores)

def Average(confidence_scores): 
    return sum(confidence_scores) / len(confidence_scores) 

overall = Average(confidence_scores)
print(overall)

[0.999353, 0.92696726, 0.9998171]
0.9753791093826294
